In [39]:
import pandas as pd
import numpy as np
import json
import pandas as pd
import numpy as np
import re
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
print('  - re=' + re.__version__)
print('  - numpy=' + np.__version__)
print('  - json=' + json.__version__)
print('  - requests=' + requests.__version__)
print('  - seaborn=' + sns.__version__)

  - re=2.2.1
  - numpy=1.19.2
  - json=2.0.9
  - requests=2.24.0
  - seaborn=0.11.0


In [22]:
clusters = pd.read_csv('../data/clusters.csv')
clustersByType = pd.read_csv('../data/clusters_by_type.csv')
violations= pd.read_csv('../data/violations.csv')

In [23]:
clusters.head()

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.125891,-86.822863
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.8467716
2,Religious Retreat,Social Gathering,3/25/2020,18,NaN,NaN
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.8195333
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.7567485


In [24]:
clusters.columns = ['name', 'facilityType', 'clusterStartDate', 'caseCount','lat','long']

In [25]:
clusters

,name,facilityType,clusterStartDate,caseCount,lat,long
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.125891,-86.822863
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.8467716
2,Religious Retreat,Social Gathering,3/25/2020,18,NaN,NaN
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.8195333
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.7567485
...,...,...,...,...,...,...
57,One Stone Church Service,Church,9/22/2020,10,36.189960,-86.7673379
58,Miss Kelli's,Bar,10/1/2020,14,36.164369,-86.7807037
59,Nashville Rescue Mission - Women's Campus,Congregate Living,10/1/2020,70,36.167693,-86.7957557
60,Link Systems Electric,Other,10/16/2020,12,36.092924,-86.7438727


In [26]:
clustersByType.head()

,Cluster Type,Number of Clusters
0,Bar,7
1,Church,2
2,College / University,5
3,Commercial-Warehouse,12
4,Congregate Living,13


In [27]:
clustersByType.columns = ['clusterType', 'clusterCount']

In [28]:
clustersByType

,clusterType,clusterCount
0,Bar,7
1,Church,2
2,College / University,5
3,Commercial-Warehouse,12
4,Congregate Living,13
5,Construction,19
6,Correctional Facility,9
7,Day Care,7
8,Food Production,2
9,Grocery,1


In [29]:
clustersGeo = clusters[clusters['lat'].isnull()==False]

In [30]:
clustersGeo.reset_index(drop=True)

,name,facilityType,clusterStartDate,caseCount,lat,long
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.125891,-86.822863
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.8467716
2,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.8195333
3,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.7567485
4,Tyson Foods,Commercial-Warehouse,4/6/2020,280,36.198993,-89.8367566
5,Nashville Center for Rehab and Healing,LTCF,4/8/2020,12,36.134557,-86.7831177
6,CDM Jail,Correctional Facility,4/13/2020,22,36.088200,-86.6861683
7,Bordeaux,LTCF,4/15/2020,47,36.180317,-86.8504781
8,Cargill,Commercial-Warehouse,4/21/2020,22,36.112551,-86.759455
9,Bethany Center for Rehab and Healing,LTCF,4/27/2020,133,36.049614,-86.7189497


In [10]:
clusters[clusters['facilityType'] =='LTCF']

,name,facilityType,clusterStartDate,caseCount,lat,long
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.8195333
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.7567485
6,Nashville Center for Rehab and Healing,LTCF,4/8/2020,12,36.134557,-86.7831177
8,Bordeaux,LTCF,4/15/2020,47,36.180317,-86.8504781
10,Bethany Center for Rehab and Healing,LTCF,4/27/2020,133,36.049614,-86.7189497
11,Knowles Assisted Living,LTCF,4/27/2020,41,36.181554,-86.8511166
16,Good Samaritan Health & Healing,LTCF,6/3/2020,75,36.053331,-86.6514237
18,The Opal at Music City,LTCF,6/3/2020,27,36.161851,-86.6632187
21,Creekside Center for Rehab and Healing,LTCF,6/15/2020,107,36.251458,-86.7286169
24,West Meade Place,LTCF,6/24/2020,11,36.089531,-86.8907533


In [31]:
violations

,case_number,status,case_subrequest,case_origin,closed_when_created,incident_address,incident_city,incident_council_district,incident_zip_code,latitude,longitude,mapped_location,date_time_closed,contact_type,parent_case
0,497255,Assigned,COVID-19 Violations,Phone,False,440 McMurray Dr,NASHVILLE,27.0,37211.0,36.061900,-86.718060,"{'latitude': '36.0619', 'longitude': '-86.71806'}",NaN,NaN,NaN
1,497616,New,COVID-19 Violations,hubNashville Community,False,8027 E Lamar Alexander Pkwy,NaN,NaN,NaN,35.679770,-83.736708,"{'latitude': '35.67977', 'longitude': '-83.736...",NaN,NaN,NaN
2,497449,Assigned,COVID-19 Violations,Phone,False,3706 Hillsboro Pike,NASHVILLE,25.0,37215.0,36.109800,-86.810920,"{'latitude': '36.1098', 'longitude': '-86.81092'}",NaN,NaN,NaN
3,497139,New,COVID-19 Violations,Phone,False,1124 Murfreesboro Rd,NaN,NaN,NaN,35.915226,-86.835418,"{'latitude': '35.9152262', 'longitude': '-86.8...",NaN,NaN,NaN
4,497019,New,COVID-19 Violations,Phone,False,5106 Old Hickory Blvd,NASHVILLE,1.0,37218.0,36.212190,-86.920010,"{'latitude': '36.21219', 'longitude': '-86.920...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11671,336829,Closed,COVID-19 Violations,Phone,False,3049 Dickerson Pike,NASHVILLE,8.0,37207.0,36.228899,-86.758057,"{'latitude': '36.22889910000001', 'longitude':...",2020-04-01T18:40:02.000,NaN,NaN
11672,336732,Closed,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-01T19:00:49.000,NaN,NaN
11673,336221,New,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11674,336157,New,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
violations[violations['latitude'].isnull()]

,case_number,status,case_subrequest,case_origin,closed_when_created,incident_address,incident_city,incident_council_district,incident_zip_code,latitude,longitude,mapped_location,date_time_closed,contact_type,parent_case
220,491272,Closed,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-30T14:00:14.000,NaN,NaN
981,471037,Closed,COVID-19 Violations,Phone,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-28T16:33:16.000,NaN,NaN
1096,474231,Closed,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-05T13:57:38.000,NaN,NaN
1434,461615,Closed,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-11T13:12:44.000,NaN,NaN
1716,454737,Closed,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-31T13:37:26.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11670,336836,Closed,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-01T18:44:31.000,NaN,NaN
11672,336732,Closed,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-01T19:00:49.000,NaN,NaN
11673,336221,New,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11674,336157,New,COVID-19 Violations,hubNashville Community,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
violationsGeo = violations[violations['latitude'].isnull()==False]

In [38]:
violationsGeo.reset_index(drop=True)

,case_number,status,case_subrequest,case_origin,closed_when_created,incident_address,incident_city,incident_council_district,incident_zip_code,latitude,longitude,mapped_location,date_time_closed,contact_type,parent_case
0,497255,Assigned,COVID-19 Violations,Phone,False,440 McMurray Dr,NASHVILLE,27.0,37211.0,36.061900,-86.718060,"{'latitude': '36.0619', 'longitude': '-86.71806'}",NaN,NaN,NaN
1,497616,New,COVID-19 Violations,hubNashville Community,False,8027 E Lamar Alexander Pkwy,NaN,NaN,NaN,35.679770,-83.736708,"{'latitude': '35.67977', 'longitude': '-83.736...",NaN,NaN,NaN
2,497449,Assigned,COVID-19 Violations,Phone,False,3706 Hillsboro Pike,NASHVILLE,25.0,37215.0,36.109800,-86.810920,"{'latitude': '36.1098', 'longitude': '-86.81092'}",NaN,NaN,NaN
3,497139,New,COVID-19 Violations,Phone,False,1124 Murfreesboro Rd,NaN,NaN,NaN,35.915226,-86.835418,"{'latitude': '35.9152262', 'longitude': '-86.8...",NaN,NaN,NaN
4,497019,New,COVID-19 Violations,Phone,False,5106 Old Hickory Blvd,NASHVILLE,1.0,37218.0,36.212190,-86.920010,"{'latitude': '36.21219', 'longitude': '-86.920...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11393,336444,Closed,COVID-19 Violations,Phone,False,1211 MEDICAL CENTER DR,NASHVILLE,18.0,37212.0,36.147020,-86.800760,"{'latitude': '36.14702', 'longitude': '-86.800...",2020-04-03T19:44:33.000,NaN,NaN
11394,336454,Closed,COVID-19 Violations,Phone,False,4105 Lebanon Pike,HERMITAGE,11.0,37076.0,36.198707,-86.620003,"{'latitude': '36.1987069', 'longitude': '-86.6...",2020-04-03T20:04:54.000,NaN,NaN
11395,336440,Closed,COVID-19 Violations,Phone,False,1045 Rev Dr Enoch Jones Blvd,NASHVILLE,19.0,37208.0,36.172200,-86.797410,"{'latitude': '36.1722', 'longitude': '-86.79741'}",2020-04-03T19:40:24.000,NaN,NaN
11396,336724,Closed,COVID-19 Violations,Phone,False,8159 Sawyer Brown Rd,NASHVILLE,22.0,37221.0,36.084467,-86.943913,"{'latitude': '36.0844668', 'longitude': '-86.9...",2020-04-01T19:01:51.000,NaN,NaN
